# Extra Model3d

In [ ]:
import magpylib as magpy
import numpy as np
from functools import partial
import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import display

class myCuboid(magpy.magnet.Cuboid):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.offset = np.array([0.,0.,0.])
        self.style.model3d.add_trace(
            backend='plotly',
            trace=partial(self.get_scatter_trace, backend='plotly'),
            makedefault=True,
            show=True
        )
        self.style.model3d.add_trace(
            backend='plotly',
            trace=partial(self.get_scatter_trace, backend='plotly'),
            makedefault=True,
            show=True,
            coordsargs={'x':'xs', 'y':'ys', 'z':'zs'},
        )

    def get_scatter_trace(self, backend):
        dim = self.dimension
        offset = self.offset
        trace =  {
                'type': 'scatter3d',
                'x': np.array([-1, -1,  1,  1, -1, -1, -1, -1, -1,  1,  1,  1,  1, 1,  1, -1])/2 * dim[0]+offset[0], 
                'y': np.array([-1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1])/2 * dim[1]+offset[1], 
                'z': np.array([-1, -1, -1, -1, -1,  1,  1, -1,  1,  1, -1,  1,  1, -1,  1,  1])/2 * dim[2]+offset[2],
                'mode':'lines',
                'line_color': f"rgb{tuple(self.magnetization.astype(int))}"
            }
        if backend == 'plotly':
            return trace
        elif backend == 'matplotlib':
            trace = {
                'type':'plot',
                **{f'{k}s': trace[k] for k in 'xyz'},
            }
            return trace

cuboid = myCuboid(magnetization=(1,0,0), dimension=(3, 3 ,3), position=(10,0,0))\
    .rotate_from_angax(np.linspace(0, 180, 36, endpoint=False), "z", anchor=(0, 0, 0), start=0)

out = widgets.Output()

@out.capture(wait=True, clear_output=True)
def update_fig(size_x, size_y, size_z, mag_x, mag_y, mag_z,  off_x, off_y, off_z, makedefault, animation):
    cuboid.dimension = size_x, size_y, size_z
    cuboid.magnetization = mag_x, mag_y, mag_z
    cuboid.offset = off_x, off_y, off_z
    for t in cuboid.style.model3d.data:
        t.makedefault = makedefault
    fig = go.Figure()
    magpy.show(cuboid, canvas=fig, animation=animation, zoom=0.5, backend='plotly')
    fig.update_layout(height=500, margin_t=10)
    fig.show()
    
wd = dict(
    size_x=widgets.FloatSlider(min=0, max=10, value=5),
    size_y=widgets.FloatSlider(min=0, max=10, value=5),
    size_z=widgets.FloatSlider(min=0, max=10, value=5),
    mag_x=widgets.IntSlider(min=0, max=255, value=0),
    mag_y=widgets.IntSlider(min=0, max=255, value=0),
    mag_z=widgets.IntSlider(min=0, max=255, value=0),
    off_x=widgets.IntSlider(min=-10, max=10, value=0),
    off_y=widgets.IntSlider(min=-10, max=10, value=0),
    off_z=widgets.IntSlider(min=-10, max=10, value=0),
    makedefault=widgets.Checkbox(value=True),
    animation=widgets.Checkbox(value=False),
)
for w in wd.values():
    w.continuous_update = False
w = widgets.interactive(update_fig, **wd)
w.update()
display(widgets.HBox([w, out]))